In [56]:
import re
import time
import requests

import numpy as np
import pandas as pd

from collections import Counter
from scrapy.http import TextResponse
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

1) Info on top taxpayers in provided on https://www.harkatu.am/ in a JavaScript gen erated table.
    
a) scrape the paid taxes, organization name and industry for top 200 taxpayers (last Quarter).<br>
b) what is the average tax paid?<br>
c) what is the top industry in the scraped list?

In [57]:
url='https://www.harkatu.am/companies/hy?0'

In [58]:
browser=webdriver.Chrome() # testing
browser.get(url)
time.sleep(2)
page = browser.page_source
browser.close()

In [59]:
print(page)

<html><head><script async="" src="//www.google-analytics.com/analytics.js"></script><script type="text/javascript" src="../wicket/resource/org.apache.wicket.resource.JQueryResourceReference/jquery/jquery-1.11.0.min-ver-8FC25E27D42774AEAE6EDBC0A18B72AA.js"></script>
<script type="text/javascript" src="../wicket/resource/org.apache.wicket.ajax.AbstractDefaultAjaxBehavior/res/js/wicket-event-jquery.min-ver-F070EDFAD7E5A1BD9662F385470094FF.js"></script>
<script type="text/javascript" src="../wicket/resource/org.apache.wicket.ajax.AbstractDefaultAjaxBehavior/res/js/wicket-ajax-jquery.min-ver-5FD38B9489AC6EA76009623682BEFC76.js"></script>
<script type="text/javascript" id="wicket-ajax-base-url">
/*<![CDATA[*/
Wicket.Ajax.baseUrl="companies/hy?0";
/*]]>*/
</script>
<script type="text/javascript" id="hiddenComponent_JS">
/*<![CDATA[*/
Wicket.Event.add(window, 'load', function (event) {     tokenManagerSkipEvent = false;function locationHashChanged() {if (tokenManagerSkipEvent) {     tokenManag

In [60]:
response = TextResponse(url=url, body=page, encoding="utf-8")

In [61]:
tax_info = response.css("div[class='volume']::text").extract()

In [62]:
print(tax_info)

['1', '13,531,276', '13,570,605', '2', '12,126,039', '12,830,020', '3', '6,300,316', '6,684,445', '4', '4,544,427', '4,544,427', '5', '3,953,688', '4,295,157', '6', '3,150,072', '3,955,956', '7', '3,028,579', '3,769,765', '8', '3,740,465', '3,753,700', '9', '2,957,433', '3,414,762', '10', '2,649,726', '2,794,111', '11', '2,590,033', '2,590,033', '12', '1,646,242', '2,389,817', '13', '2,134,245', '2,356,659', '14', '1,630,605', '2,061,793', '15', '1,751,630', '1,751,630', '16', '1,273,390', '1,564,916', '17', '425,777', '1,485,612', '18', '846,454', '1,403,194', '19', '1,353,765', '1,399,955', '20', '1,344,333', '1,363,849', '21', '87,891', '1,316,943', '22', '1,273,536', '1,279,783', '23', '1,276,919', '1,276,919', '24', '2,863', '1,262,660', '25', '1,236,542', '1,261,087', '177,072,192', '221,020,204']


In [67]:
tax_info[2]

'13,570,605'

In [68]:
companies = response.css("h1>a>span::text").extract()

In [69]:
print(companies)

['ԳԱԶՊՐՈՄ ԱՐՄԵՆԻԱ', 'ԳՐԱՆԴ ՏՈԲԱԿՈ', 'ԶԱՆԳԵԶՈՒՐԻ ՊՂՆՁԱՄՈԼԻԲԴԵՆԱՅԻՆ ԿՈՄԲԻՆԱՏ', 'ՀԱՅԱՍՏԱՆԻ ԷԼԵԿՏՐԱԿԱՆ ՑԱՆՑԵՐ', 'ԳԵՈՊՐՈՄԱՅՆԻՆԳ ԳՈԼԴ', 'ԻՆՏԵՐՆԵՅՇՆԼ ՄԱՍԻՍ ՏԱԲԱԿ', 'ՍԻՓԻԷՍ ՕԻԼ ՔՈՐՓՈՐԵՅՇՆ', 'ՖԼԵՇ', 'ԱԿՎԻԼՈՆ ԹՐԵՅԴ', 'ԱՐՄԵՆԻԱ ՄԻՋԱԶԳԱՅԻՆ ՕԴԱՆԱՎԱԿԱՅԱՆՆԵՐ', 'ՄԱՔՍ ՊԵՏՐՈԼ', 'ՀԱՅԿԱԿԱՆ ԱՏՈՄԱՅԻՆ ԷԼԵԿՏՐԱԿԱՅԱՆ', 'ԱՐՄԵՆԻԱՆ ՄԱՅՆԻՆԳ ՔՈՆԹՐԱՔԹՈՐ', 'ՅՈՒՔՈՄ', 'ԳԱԶՊՐՈՄ ԱՐՄԵՆԻԱ ՓԲԸ ՏՐԱՆՍԳԱԶ', 'ՍԻԹԻ', 'ԳՐԱՆԴ ՄԱՍՏԵՐ', 'ԳՐԱՆԴ ՔԵՆԴԻ', 'ՄՈԲԱՅԼ ՍԵՆԹՐ ԱՐԹ', 'ԱՄԵՐԻԱԲԱՆԿ', 'ԱԼԵՔՍ ԷՆԴ ՀՈԼԴԻՆԳ', 'ՎԵՈԼԻԱ ՋՈՒՐ', 'ՀԱՅԲԻԶՆԵՍԲԱՆԿ ՓԲԸ', 'ԱՎՏՈՆԵՐԿՐՈՂ -10', 'ԵՐԵՎԱՆԻ ՋԵՐՄԱԷԼԵԿՏՐԱԿԵՆՏՐՈՆ']


In [70]:
industries = response.css("div[class='industry']::text").extract()

In [71]:
print(industries)

['Գազի մատակարարում', 'Ծխախոտային արտադրատեսակներ', 'Մետաղական հանքարդյունաբերություն', 'Էլեկտրականության մատակարարում', 'Մետաղական հանքարդյունաբերություն', 'Ծխախոտային արտադրատեսակներ', 'Վառելիքի վաճառք, Վառելիքի ներմուծում', 'Մեծածախ առևտուր, Մանրածախ առևտուր', 'Հագուստի վաճառք, Մեծածախ առևտուր', 'Տրանսպորտ', 'Վառելիքի վաճառք, Վառելիքի ներմուծում', 'Էլեկտրականության արտադրություն', 'Ցամաքային տրանսպորտ, Տրանսպորտ', 'Հեռահաղորդակցություն', 'Գազի մատակարարում', 'Արդյունաբերություն', 'Սննդամթերքի արտադրություն', 'Ներմուծում, Առևտուր', 'Բանկային ոլորտ', 'Ջրամատակարարում և կոյուղի', 'Բանկային ոլորտ', 'Էլեկտրականության արտադրություն']


In [84]:
def company_scraper(url):
    browser.get(url)
    time.sleep(2)
    page = browser.page_source
    response = TextResponse(url=url, body=page, encoding="utf-8")
    companies = response.css("h1>a>span::text").extract()
    return companies

In [85]:
browser=webdriver.Chrome()
all_companies=[] 
for i in range(8):
    url = "https://www.harkatu.am/companies/hy?0#p/" + str(i+1) 
    companies = company_scraper(url)
    all_companies.extend(companies) 
    time.sleep(2)
print(all_companies)
browser.close()

['ԳԱԶՊՐՈՄ ԱՐՄԵՆԻԱ', 'ԳՐԱՆԴ ՏՈԲԱԿՈ', 'ԶԱՆԳԵԶՈՒՐԻ ՊՂՆՁԱՄՈԼԻԲԴԵՆԱՅԻՆ ԿՈՄԲԻՆԱՏ', 'ՀԱՅԱՍՏԱՆԻ ԷԼԵԿՏՐԱԿԱՆ ՑԱՆՑԵՐ', 'ԳԵՈՊՐՈՄԱՅՆԻՆԳ ԳՈԼԴ', 'ԻՆՏԵՐՆԵՅՇՆԼ ՄԱՍԻՍ ՏԱԲԱԿ', 'ՍԻՓԻԷՍ ՕԻԼ ՔՈՐՓՈՐԵՅՇՆ', 'ՖԼԵՇ', 'ԱԿՎԻԼՈՆ ԹՐԵՅԴ', 'ԱՐՄԵՆԻԱ ՄԻՋԱԶԳԱՅԻՆ ՕԴԱՆԱՎԱԿԱՅԱՆՆԵՐ', 'ՄԱՔՍ ՊԵՏՐՈԼ', 'ՀԱՅԿԱԿԱՆ ԱՏՈՄԱՅԻՆ ԷԼԵԿՏՐԱԿԱՅԱՆ', 'ԱՐՄԵՆԻԱՆ ՄԱՅՆԻՆԳ ՔՈՆԹՐԱՔԹՈՐ', 'ՅՈՒՔՈՄ', 'ԳԱԶՊՐՈՄ ԱՐՄԵՆԻԱ ՓԲԸ ՏՐԱՆՍԳԱԶ', 'ՍԻԹԻ', 'ԳՐԱՆԴ ՄԱՍՏԵՐ', 'ԳՐԱՆԴ ՔԵՆԴԻ', 'ՄՈԲԱՅԼ ՍԵՆԹՐ ԱՐԹ', 'ԱՄԵՐԻԱԲԱՆԿ', 'ԱԼԵՔՍ ԷՆԴ ՀՈԼԴԻՆԳ', 'ՎԵՈԼԻԱ ՋՈՒՐ', 'ՀԱՅԲԻԶՆԵՍԲԱՆԿ ՓԲԸ', 'ԱՎՏՈՆԵՐԿՐՈՂ -10', 'ԵՐԵՎԱՆԻ ՋԵՐՄԱԷԼԵԿՏՐԱԿԵՆՏՐՈՆ', 'ՉԻՄԵԳ', 'ՕՆԻՐԱ ՔԼԱԲ', 'ՍԵԼԵՆԱ', 'ԱՐԴՇԻՆԲԱՆԿ', 'ՆԱՏԱԼԻ ՖԱՐՄ', 'ԻՄԷՔՍ ԳՐՈՒՊ', 'ՖԻԼԻՊ ՄՈՐՐԻՍ ԱՐՄԵՆԻԱ', 'ԱԿԲԱ-ԿՐԵԴԻՏ ԱԳՐԻԿՈԼ ԲԱՆԿ', 'ԿՈԿԱ-ԿՈԼԱ ՀԵԼԼԵՆԻԿ ԲՈԹԼԻՆԳ ՔԱՄՓՆԻ ԱՐՄԵՆԻԱ', 'ՋԵՐՄՈՒԿ ԳՐՈՒՊ', 'ՀԱՐԱՎԿՈՎԿԱՍՅԱՆ ԵՐԿԱԹՈՒՂԻ', 'ԻՆԵԿՈԲԱՆԿ', 'ՍԱՍ-ԳՐՈՒՊ', 'ԲԱՐՍԻՍ', 'ՎԵԳԱ ՈՒՈՐԼԴ', 'ՋԵՅ ԹԻ ԱՅ ԱՐՄԵՆԻԱ', 'ՎԻԷԼՎԻ ՍԵՆԹՐ', 'ԿՈՆՎԵՐՍ ԲԱՆԿ', 'ՍԻՆՈՓՍԻՍ ԱՐՄԵՆԻԱ', 'ԲԱՂՐԱՄՅԱՆ', 'ՍՊԱՅԿԱ', 'ԴԵԿՈՐԱ-ԳՐՈՒՊ', 'ԵՐԵՎԱՆԻ ԿՈՆՅԱԿԻ ԳՈՐԾԱՐԱՆ', 'ԵՐԵՎԱՆԻ ՊԵՏԱԿԱՆ ՀԱՄԱԼՍԱՐԱՆ', 'ՎԱԳԱ-ՖԱՐՄ', 

In [86]:
len(all_companies)

200

In [87]:
def tax_scraper(url):
    browser.get(url)
    time.sleep(2)
    page = browser.page_source
    response = TextResponse(url=url, body=page, encoding="utf-8")
    tax_info = response.css("div[class='volume']::text").extract()
    taxes = []
    i = 2
    while i < len(tax_info):
        taxes.append(tax_info[i])
        i = i + 3 # as information we need is on each third position, we implement while loop
    return taxes

In [77]:
browser=webdriver.Chrome()
all_taxes=[] 
for i in range(8):
    url = "https://www.harkatu.am/companies/hy?0#p/" + str(i+1) 
    taxes = tax_scraper(url)
    all_taxes.extend(taxes) 
    time.sleep(2) 
print(all_taxes)
browser.close()

['13,570,605', '12,830,020', '6,684,445', '4,544,427', '4,295,157', '3,955,956', '3,769,765', '3,753,700', '3,414,762', '2,794,111', '2,590,033', '2,389,817', '2,356,659', '2,061,793', '1,751,630', '1,564,916', '1,485,612', '1,403,194', '1,399,955', '1,363,849', '1,316,943', '1,279,783', '1,276,919', '1,262,660', '1,261,087', '1,220,359', '1,218,400', '1,164,814', '1,128,300', '1,112,962', '1,099,085', '1,096,554', '1,069,783', '1,061,270', '1,046,473', '1,043,767', '1,010,827', '1,004,964', '960,942', '911,332', '862,030', '844,676', '841,605', '834,277', '785,707', '784,674', '770,479', '738,037', '722,658', '712,742', '711,728', '696,962', '693,692', '684,939', '676,624', '666,851', '653,659', '648,152', '645,610', '645,503', '633,138', '632,741', '631,944', '630,496', '613,588', '604,836', '601,638', '593,542', '579,473', '558,835', '558,030', '545,105', '544,689', '541,690', '533,199', '533,038', '528,902', '528,044', '519,370', '501,449', '496,933', '487,964', '483,581', '477,682

In [78]:
len(all_taxes)

200

In [79]:
taxes_clean = np.array([int(all_taxes[i].replace(",","")) for i in range(len(all_taxes))])

In [80]:
print(taxes_clean)

[13570605 12830020  6684445  4544427  4295157  3955956  3769765  3753700
  3414762  2794111  2590033  2389817  2356659  2061793  1751630  1564916
  1485612  1403194  1399955  1363849  1316943  1279783  1276919  1262660
  1261087  1220359  1218400  1164814  1128300  1112962  1099085  1096554
  1069783  1061270  1046473  1043767  1010827  1004964   960942   911332
   862030   844676   841605   834277   785707   784674   770479   738037
   722658   712742   711728   696962   693692   684939   676624   666851
   653659   648152   645610   645503   633138   632741   631944   630496
   613588   604836   601638   593542   579473   558835   558030   545105
   544689   541690   533199   533038   528902   528044   519370   501449
   496933   487964   483581   477682   472185   471512   449171   440455
   438385   433932   433820   416144   406454   405745   405263   401719
   395131   395082   389125   384375   533038   528902   528044   519370
   501449   496933   487964   483581   477682   472

In [81]:
avg_tax = np.mean(taxes_clean)

In [82]:
print(avg_tax) #average tax

828040.17


In [88]:
def industry_scraper(url):
    browser.get(url)
    time.sleep(2) #to wait for loading
    page = browser.page_source
    response = TextResponse(url=url, body=page, encoding="utf-8")
    industry = response.css("div[class='industry']::text").extract()
    time.sleep(2)
    return industry

In [89]:
browser=webdriver.Chrome()
all_industries=[] 
for i in range(8):
    url = "https://www.harkatu.am/companies/hy?0#p/" + str(i+1) 
    current_page_industries = industry_scraper(url)
    all_industries.extend(current_page_industries) 
    time.sleep(2) 
print(all_industries)
browser.close()

['Գազի մատակարարում', 'Ծխախոտային արտադրատեսակներ', 'Մետաղական հանքարդյունաբերություն', 'Էլեկտրականության մատակարարում', 'Մետաղական հանքարդյունաբերություն', 'Ծխախոտային արտադրատեսակներ', 'Վառելիքի վաճառք, Վառելիքի ներմուծում', 'Մեծածախ առևտուր, Մանրածախ առևտուր', 'Մեծածախ առևտուր, Հագուստի վաճառք', 'Տրանսպորտ', 'Վառելիքի ներմուծում, Վառելիքի վաճառք', 'Էլեկտրականության արտադրություն', 'Տրանսպորտ, Ցամաքային տրանսպորտ', 'Հեռահաղորդակցություն', 'Գազի մատակարարում', 'Արդյունաբերություն', 'Սննդամթերքի արտադրություն', 'Ներմուծում, Առևտուր', 'Բանկային ոլորտ', 'Ջրամատակարարում և կոյուղի', 'Բանկային ոլորտ', 'Էլեկտրականության արտադրություն', 'Սննդամթերքի և խմիչքի վաճառք', 'Խաղատների գործունեություն', 'Բանկային ոլորտ', 'Դեղատներ', 'Շինանյութի ներմուծում', 'Ծխախոտային ապրանքների ներմուծում', 'Բանկային ոլորտ', 'Խմիչքների արտադրություն', 'Խմիչքների արտադրություն', 'Ցամաքային տրանսպորտ', 'Բանկային ոլորտ', 'Մանրածախ առևտուր, Սննդամթերքի և խմիչքի վաճառք', 'Ներմուծում', 'Էլեկտրատեխնիկայի վաճառք, Կահույքի

In [90]:
len(all_industries)

183

In [91]:
counter = Counter(all_industries).most_common()

In [92]:
counter[0] #most common industry is banking sector (Բանկային ոլորտ)

('Բանկային ոլորտ', 14)